In [24]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import numpy as np

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [25]:
#训练数据
X=['Chinese Beijing Chinese', 'Chinese Chinese Shanghai', 'Chinese Macao',  'Tokyo Japan Chinese']
y = ['yes','yes','yes','no']

In [26]:
#以词频为特征向量化
cv = CountVectorizer().fit(X)
X =cv.transform(X)

In [32]:
print ('词典：',sorted(cv.vocabulary_.items(), key=lambda d: d[1]))
X.toarray()

词典： [('beijing', 0), ('chinese', 1), ('japan', 2), ('macao', 3), ('shanghai', 4), ('tokyo', 5)]


array([[1, 2, 0, 0, 0, 0],
       [0, 2, 0, 0, 1, 0],
       [0, 1, 0, 1, 0, 0],
       [0, 1, 1, 0, 0, 1]], dtype=int64)

In [28]:
clf = MultinomialNB()
clf.fit(X,y)
clf.class_count_ #类计数
clf.feature_count_  #特征计数
clf.class_log_prior_ #类权重 log
clf.intercept_

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

array([ 1.,  3.])

array([[ 0.,  1.,  1.,  0.,  0.,  1.],
       [ 1.,  5.,  0.,  1.,  1.,  0.]])

array([-1.38629436, -0.28768207])

array([-0.28768207])

In [33]:

alpha = 1 #平滑
smoothed_fc = clf.feature_count_ + alpha
smoothed_cc = smoothed_fc.sum(axis=1)

feature_log_prob_ = (np.log(smoothed_fc) -
                                  np.log(smoothed_cc.reshape(-1, 1)))
fp = smoothed_fc / smoothed_cc.reshape(2,1) #特征概率

smoothed_fc
smoothed_cc
fp
np.e ** feature_log_prob_

array([[ 1.,  2.,  2.,  1.,  1.,  2.],
       [ 2.,  6.,  1.,  2.,  2.,  1.]])

array([  9.,  14.])

array([[ 0.11111111,  0.22222222,  0.22222222,  0.11111111,  0.11111111,
         0.22222222],
       [ 0.14285714,  0.42857143,  0.07142857,  0.14285714,  0.14285714,
         0.07142857]])

array([[ 0.11111111,  0.22222222,  0.22222222,  0.11111111,  0.11111111,
         0.22222222],
       [ 0.14285714,  0.42857143,  0.07142857,  0.14285714,  0.14285714,
         0.07142857]])

In [34]:
#预测
x_val = cv.transform(['Chinese Chinese Chinese Tokyo Japan']) #训练样本，向量化
x_val.toarray()[0]  #
clf.predict_proba(x_val) #模型预测概率

array([0, 3, 1, 0, 0, 1], dtype=int64)

array([[ 0.31024139,  0.68975861]])

In [58]:
smoothed_fc

array([[ 1.,  2.,  2.,  1.,  1.,  2.],
       [ 2.,  6.,  1.,  2.,  2.,  1.]])

In [61]:
#手工计算概率
PX = smoothed_fc.sum(axis=0) / smoothed_fc.sum()

rst=[0,0]
for c in [0,1]:
    p = 1
    for i,x in enumerate(x_val.toarray()[0]):
        p *= (fp[c,i]) ** x 
    p *=  (np.e ** clf.class_log_prior_[c]) 
    
    rst[c] = p
    print (c,p)


#按比例归一
rst[0] *1/(rst[0]+rst[1])
rst[1] *1/(rst[0]+rst[1])

0 0.189236169913
1 0.420728125918


0.31024138823653269

0.68975861176346742